In [1]:
import time
import pandas as pd
import os
import os.path
import json
import sqlite3

In [8]:
db_name = "./UPS2_DIA_180907/UPS2_DIA_180907.sqlite"
db_conn = sqlite3.connect(db_name)
processing_info_df = pd.read_sql_query("select value from processing_info where item='feature_batch_info'", db_conn)
db_conn.close()

In [20]:
json.loads(processing_info_df.iloc[0].item())[0]['db']

u'./UPS2_DIA_180907/UPS2_DIA_180907-features-1-100.sqlite'

In [21]:
len(json.loads(processing_info_df.iloc[0].item()))

48

In [25]:
databases = json.loads(processing_info_df.iloc[0].item())

In [26]:
databases[0]['db']

u'./UPS2_DIA_180907/UPS2_DIA_180907-features-1-100.sqlite'

In [28]:
feature_list_csv = "feature_list.csv"
deconvoluted_ions_csv = "deconvoluted_ions.csv"
feature_list_df = None
deconvoluted_ions_df = None
for db in databases:
    db_conn = sqlite3.connect(db['db'])
    # load the feature list table
    df = pd.read_sql_query("select * from feature_list order by feature_id", db_conn)
    if feature_list_df is None:
        feature_list_df = df.copy()
    else:
        feature_list_df = feature_list_df.append(df, ignore_index=True)
    # load the deconvoluted ions table
    df = pd.read_sql_query("select * from deconvoluted_ions order by feature_id", db_conn)
    if deconvoluted_ions_df is None:
        deconvoluted_ions_df = df.copy()
    else:
        deconvoluted_ions_df = deconvoluted_ions_df.append(df, ignore_index=True)
    db_conn.close()
# write out the DFs as CSVs
feature_list_df.to_csv(feature_list_csv, mode='w', sep=',', index=False, header=True)
deconvoluted_ions_df.to_csv(deconvoluted_ions_csv, mode='w', sep=',', index=False, header=True)